In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as Func
import numpy as np
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.io import savemat 
import os
from sklearn.preprocessing import normalize

In [2]:
def make_dataset(root):
    dataset = []
    with open(root) as f:
        #print(root)
        content = f.read().splitlines()
    for line in content:
        #path = np.load(line)
        dataset.append(line)
    return dataset
class lateSpeech(data.Dataset):
    def __init__(self,root1,root2):
        self.data1 = make_dataset(root1)
        #print(self.data1[0])
        self.data2 = make_dataset(root2)

        self.root1 = root1
        self.root2 = root2


    def __getitem__(self, index):
        
        Xfile1 = self.data1[index]

        de_file = self.data2[index]
        X1 = np.load(Xfile1)
        X1 = normalize(X1)
        de = np.load(de_file)
        

        return torch.from_numpy(X1).t().float(),torch.from_numpy(de).float() 

    def __len__(self):
        return len(self.data1)



In [3]:
def InputSize(x):
    a = np.load(x)
    b = a.shape
    return b[0]
def seqLen(x):
    a = np.load(x)
    b = a.shape
    return b[1]


In [4]:
input_file = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb_train/train_wlen_80_nfft_128_overlap_10/reverb_rir5000_roomNum10_t600.9_loc500_8kHz.npy'
new_file = '/data/liyuy/PROJECTS/DEREVERB3/timit_8k/reverb_valid/valid_wlen_80_nfft_128_overlap_10/reverb_rir0500_roomNum10_t600.9_loc500_8kHz.npy'
print(seqLen(new_file),seqLen(input_file))

class param:
    #img_size = (80, 80)
    bs = 20
    lr = 10e-4
    epochs = 100
    hsize = 513
    hlayer = 6
    osize = 1026
    lstm_s = InputSize(input_file)
    lstm_l = seqLen(input_file)
    ts = 1

mix1_train = "/data/liyuy/PROJECTS/DEREVERB3/block_conv/stft_address/train/train_wlen_80_nfft_128_overlap_10.txt"

de_train = "/data/liyuy/PROJECTS/DEREVERB3/block_conv/target_address/train/train_target.txt"

mix1_val = "/data/liyuy/PROJECTS/DEREVERB3/block_conv/stft_address/valid/valid_wlen_80_nfft_128_overlap_10.txt"

de_val = "/data/liyuy/PROJECTS/DEREVERB3/block_conv/target_address/valid/valid_target.txt"

#mix1_test = "/data/liyuy/PROJECTS/DEREVERB3/block_conv/stft_address/test/test1/test_wlen_80_nfft_128_overlap_10.txt""

#de_test = "/data/liyuy/PROJECTS/DEREVERB3/block_conv/address_seg/target/testing/complex/multi/reverb_900_1_0.9.txt"


train_dl = data.DataLoader(lateSpeech(mix1_train,de_train),
                        batch_size=param.bs,
                        shuffle=True,
                        pin_memory=torch.cuda.is_available())
val_dl = data.DataLoader(lateSpeech(mix1_val,de_val),
                    batch_size=param.bs,
                    shuffle=False,
                    pin_memory=torch.cuda.is_available())

#test_dl = data.DataLoader(lateSpeech(mix1_test,de_test),
#                         batch_size=param.bs,
#                         shuffle=False)


758 758


In [10]:
class LSTM_hn(nn.Module):
    def __init__(self):
        super(LSTM_hn,self).__init__()
        self.hsize = param.hsize
        self.hlayer = param.hlayer
        self.batchSize = param.bs
        self.h0 = self.init_hidden(self.hsize,self.hlayer)
        self.c0 = self.init_cell(self.hsize,self.hlayer)
        self.lstm = nn.LSTM(param.lstm_s,self.hsize,self.hlayer,batch_first=True) 
        self.fc1 = nn.Linear(param.lstm_l*self.hsize,self.hsize)
        self.fc2 = nn.Linear(self.hsize,param.osize)
    def init_hidden(self,hidden_size,hidden_layer):
        return Variable(torch.zeros(hidden_layer,self.batchSize, hidden_size).cuda())
    
    def init_cell(self,hidden_size,hidden_layer):
        return Variable(torch.zeros(hidden_layer,self.batchSize, hidden_size).cuda())   
        
    def forward(self,sig1):
        
        hx = self.h0
        cx = self.c0
        
        out,(hx,cx) = self.lstm(sig1,(hx,cx))
        
        new_out = out.contiguous().view(-1,param.lstm_l * self.hsize)
        
        output1 = Func.relu(self.fc1(new_out))
        
        de_out = Func.leaky_relu(self.fc2(output1))
        
        
        return de_out

In [11]:
model = LSTM_hn().cuda()
def weights(m):
    if isinstance(m,nn.Linear):
        nn.init.xavier_normal_(m.weight.data)
        nn.init.constant_(m.bias.data,0.1)
    elif isinstance(m, nn.LSTM):
        for param in m.parameters():
            if len(param.shape) >= 2:
                nn.init.orthogonal_(param.data)
            else:
                nn.init.normal_(param.data) 
model.apply(weights)
optim = torch.optim.Adam(model.parameters(), lr=param.lr)

criterion = nn.MSELoss()

In [12]:
def get_loss(dl, model):
    loss = 0
    for X1, y1 in dl:
        X1, y1 = Variable(X1).cuda(), Variable(y1).cuda()
        output = model(X1)
    
        loss1 = criterion(output,y1)

        
        loss += loss1.cpu().item() * param.bs
    loss = loss / (len(val_dl.dataset))
    return loss

In [13]:
iters = []
train_losses = []
val_losses = []

it = 0
min_loss = np.inf
bst_model_fpath = '/data/liyuy/PROJECTS/DEREVERB3/LSTM/exp5/model/bst_model_wlen_80_nfft_128_overlap_10.pth'
model.train(True)

for epoch in range(1,param.epochs):
    loss = 0.0
    model.train(True)
    with torch.set_grad_enabled(True):
        for mag1,de_gtruth in train_dl:
            #print(mag.shape)
            mag1 = Variable(mag1.cuda())  # [N, 1, H, W]
            de_gtruth = Variable(de_gtruth.cuda())

            output = model(mag1)# [N, 2, H, W]
            
            

            pLoss = criterion(output,de_gtruth)

            loss += pLoss.cpu().item() * param.bs
            optim.zero_grad()
            pLoss.backward()
            optim.step()
        avgLoss = loss/len(train_dl.dataset)
      
    model.eval()
    
    val_loss = get_loss(val_dl, model)
    
     
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), bst_model_fpath)              
        print('Epoch {:2}, Train Loss:{:>.9f}, Validation Loss:{:>.9f}'.format(epoch,avgLoss,min_loss))
    print(epoch)

Epoch  1, Train Loss:0.356883924, Validation Loss:0.003124676
1
Epoch  2, Train Loss:0.002765336, Validation Loss:0.002662965
2


KeyboardInterrupt: 

In [ ]:
bst_model_fpath = '/data/liyuy/PROJECTS/DEREVERB3/LSTM/exp5/model/bst_model_wlen_80_nfft_128_overlap_10.pth'


outputPath = '/data/liyuy/PROJECTS/DEREVERB3/LSTM/exp5/output/output_de_all/test1_wlen_80_nfft_128_overlap_10/'
model.load_state_dict(torch.load(bst_model_fpath))


model.eval()
#new_count = 1
index = 1
Fs = 8e3
loss = 0.0
t60s = [0.3,0.6,0.9]
t60_i = 0   
i_loc = 1

with torch.set_grad_enabled(False):
    for mag1,de_gtruth in test_dl:
            #print(mag.shape)
        mag1 = Variable(mag1.cuda())  # [N, 1, H, W]
        
        de_gtruth = Variable(de_gtruth.cuda())
        
            # [N, H, W] with class indices (0, 1)
        output1 = model(mag1)# [N, 2, H, W]
        
        pLoss = criterion(output1,de_gtruth)
        
        loss += pLoss.cpu().item() * param.ts
        avgLoss = loss/len(test_dl.dataset)
        
 
        
        if index != 1 and (index - 1) % 3 == 0:
            t60_i = 0
            
        de_com = output1[0].cpu().data.numpy()
        de_com = np.squeeze(de_com)
            
        de_real = de_com[0]
        de_real = np.expand_dims(de_real,axis=1)
   
        de_imag = de_com[0]
        de_imag = np.expand_dims(de_imag,axis=1)
            
            
        fft_de = de_real + 1j * de_imag
            
        com_de = fft_de[1:512]
            #print(com_de.shape)
            #flip and take the complex conjugate part, and combine them together
        com_de = np.conj(np.flip(com_de))
        new_fft_de = np.concatenate((fft_de[0:512],fft_de[512],com_de),axis=0)
            
        de = np.abs(np.fft.ifft(new_fft_de))
            
        t60 = t60s[t60_i]
        outPath = outputPath +'te_de_'+ "{0:0=4d}".format(index) + '_t60' + '%f' % t60 + '_loc' + '%d' % (i_loc) +'recons.mat'
        savemat(outPath,{'de':de})
            
        index += 1
        i_loc += 1
        t60_i += 1
    print('Test Loss:{:>.9f}'.format(avgLoss))    
